In [14]:
import sys
#sys.path.append('/home/macul/libraries/MXNet2Caffe')
import pandas as pd
from collections import namedtuple
import numpy as np
import cv2
import pickle
from PIL import Image
import psutil
def process_df(df_rst):
    df_rst['user_id_top0'] = df_rst['user_id'].apply(lambda x: x[0])
    df_rst['score_top0'] = df_rst['score'].apply(lambda x: x[0])
    df_rst['score_top1'] = df_rst['score'].apply(lambda x: x[1])
    df_rst['angle_top0']=np.arccos(df_rst['score_top0'])
    df_rst['angle_top1']=np.arccos(df_rst['score_top1'])
    df_rst['dst_path_top0'] = df_rst['dst_path'].apply(lambda x: x[0])
    df_rst['dst_path_top0_name']=df_rst['dst_path_top0'].apply(lambda x: x.split('/')[-1].split('.')[0])
    df_rst['src_path_name']=df_rst['src_path'].apply(lambda x: x.split('/')[-2])
    return df_rst
def my_sort(x):
    x.sort()
    return x
def add_columns_suffix(df, suffix):
    old_columns = df.columns.tolist()
    new_columns = []
    for col in old_columns:
        if col != 'src_path':
            col = col+suffix
        new_columns.append(col)
    print(new_columns)
    df.columns = new_columns
    return df

In [65]:
df1=pd.read_pickle('/media/macul/black/face_database_raw_data/faceRecog_out_crop/faceRecogOut_match_db_insight_original.pkl')
#df2=pd.read_pickle('/media/macul/black/face_database_raw_data/faceRecog_out_crop/faceRecogOut_match_db_tf_mobilenet.pkl')
df2=pd.read_pickle('/media/macul/black/face_database_raw_data/faceRecog_out_crop/faceRecogOut_match_db_caffe_sphereface_deepglint.pkl')
#df2=pd.read_pickle('/media/macul/black/face_database_raw_data/faceRecog_out_crop/faceRecogOut_match_db_insight_baseline.pkl')
df3=pd.read_pickle('/media/macul/black/face_database_raw_data/faceRecog_out_crop/faceRecogOut_match_db_dgx_train1.pkl')
#df3=pd.read_pickle('/media/macul/black/face_database_raw_data/faceRecog_out_crop/faceRecogOut_match_db_landmarkloss_bfBugFix.pkl')
#df3=pd.read_pickle('/media/macul/black/face_database_raw_data/faceRecog_out_crop/faceRecogOut_match_db_caffe_sphereface_asianface_glass_pp.pkl')
#df3=pd.read_pickle('/media/macul/black/face_database_raw_data/faceRecog_out_crop/faceRecogOut_match_db_caffe_sphereface_asianface.pkl')
df1=process_df(df1)
df2=process_df(df2)
df3=process_df(df3)
df1=add_columns_suffix(df1, '_1')
df2=add_columns_suffix(df2, '_2')
df3=add_columns_suffix(df3, '_3')

['src_path', 'dst_path_1', 'user_id_1', 'score_1', 'user_id_top0_1', 'score_top0_1', 'score_top1_1', 'angle_top0_1', 'angle_top1_1', 'dst_path_top0_1', 'dst_path_top0_name_1', 'src_path_name_1']
['src_path', 'dst_path_2', 'user_id_2', 'score_2', 'user_id_top0_2', 'score_top0_2', 'score_top1_2', 'angle_top0_2', 'angle_top1_2', 'dst_path_top0_2', 'dst_path_top0_name_2', 'src_path_name_2']
['src_path', 'dst_path_3', 'user_id_3', 'score_3', 'user_id_top0_3', 'score_top0_3', 'score_top1_3', 'angle_top0_3', 'angle_top1_3', 'dst_path_top0_3', 'dst_path_top0_name_3', 'src_path_name_3']


In [66]:
df12=pd.merge(df1, df2, how='inner', on='src_path', copy=True, indicator=False, validate=None)
df123=pd.merge(df12, df3, how='inner', on='src_path', copy=True, indicator=False, validate=None)

In [67]:
print(len(df1))
print(len(df2))
print(len(df123))

120361
120361
120361


In [45]:
df123.keys()

Index(['src_path', 'dst_path_1', 'user_id_1', 'score_1', 'user_id_top0_1',
       'score_top0_1', 'score_top1_1', 'angle_top0_1', 'angle_top1_1',
       'dst_path_top0_1', 'dst_path_top0_name_1', 'src_path_name_1',
       'dst_path_2', 'user_id_2', 'score_2', 'user_id_top0_2', 'score_top0_2',
       'score_top1_2', 'angle_top0_2', 'angle_top1_2', 'dst_path_top0_2',
       'dst_path_top0_name_2', 'src_path_name_2', 'dst_path_3', 'user_id_3',
       'score_3', 'user_id_top0_3', 'score_top0_3', 'score_top1_3',
       'angle_top0_3', 'angle_top1_3', 'dst_path_top0_3',
       'dst_path_top0_name_3', 'src_path_name_3'],
      dtype='object')

In [80]:
print(len(df123[df123['score_top0_1']>0.5]),len(df123[df123['score_top0_2']>0.57277]),len(df123[df123['score_top0_3']>0.4601]))

33461 33467 33465


In [81]:
df123_m = df123[(df123['score_top0_1']>0.5)|(df123['score_top0_2']>0.57277)|(df123['score_top0_3']>0.4601)]

In [82]:
print(len(df123_m))

42917


In [83]:
df123_m_equal = df123_m[(df123_m['dst_path_top0_name_1']==df123_m['dst_path_top0_name_2'])&(df123_m['dst_path_top0_name_1']==df123_m['dst_path_top0_name_3'])]

In [84]:
print(len(df123_m_equal))

28197


In [95]:
df123_m_equal['score_top0_2'].min()

0.3600414991378784

In [85]:
print(df123_m_equal['score_top0_1'].min(),df123_m_equal['score_top0_2'].min(),df123_m_equal['score_top0_3'].min())

0.3336476683616638 0.3600414991378784 0.29931190609931946


In [90]:
df1_50=df1[df1['score_top0_1']>0.5]
df2_57=df2[df2['score_top0_2']>0.57277]
print(len(df1_50))
print(len(df2_57))
df12_m=df1_50.merge(df2_57, on=['src_path'])
df12_m_diff=df12_m[df12_m['dst_path_top0_name_1']!=df12_m['dst_path_top0_name_2']].reset_index()
print(len(df12_m_diff))

33461
33467
2970


In [91]:
df12_m_diff['dst_path_top0_name_diff_sorted'] = df12_m_diff.apply(lambda x: [x['dst_path_top0_name_1'],x['dst_path_top0_name_2']], axis=1)
df12_m_diff['dst_path_top0_name_diff_sorted'] = df12_m_diff['dst_path_top0_name_diff_sorted'].apply(my_sort)

In [92]:
unique_diff = []
for d in df12_m_diff['dst_path_top0_name_diff_sorted'].values.tolist():
    if d not in unique_diff:
        unique_diff.append(d)

In [93]:
same_person = []
img_loc = '/media/macul/black/face_database_raw_data/faceRecog_out_database/var/data/ego_java_server_data/IIM_Images_Crop/{}.jpg'
for i, ud in enumerate(unique_diff):
    print(i,len(unique_diff), ud)
    if ud not in same_person:
        img1=Image.open(img_loc.format(ud[0]))
        img1.show()
        img2=Image.open(img_loc.format(ud[1]))
        img2.show()
        if input()=='y':
            same_person.append(ud)

0 578 ['84e09aa32b6c4dff9693ec03baf9cc68', '8dcccd3e1b824d80b073fc2e7b731fc0']

1 578 ['8dcccd3e1b824d80b073fc2e7b731fc0', '904321946cde45499e402c605b686ab4']

2 578 ['15160cc1c82e4e959f7377ea96ae19f9', '809767d7fb8d4c60a2897a219ca6879d']
y
3 578 ['15160cc1c82e4e959f7377ea96ae19f9', 'c8cb43f0bfe043029681133a8ee6a3a5']
y
4 578 ['15160cc1c82e4e959f7377ea96ae19f9', '5c8f8560eaea4785bb6ad6f367f898d4']
y
5 578 ['5c8f8560eaea4785bb6ad6f367f898d4', 'c8cb43f0bfe043029681133a8ee6a3a5']
y
6 578 ['15160cc1c82e4e959f7377ea96ae19f9', '294df1b57d714fa087e6bb6e1fe3645a']
y
7 578 ['809767d7fb8d4c60a2897a219ca6879d', 'c8cb43f0bfe043029681133a8ee6a3a5']
y
8 578 ['294df1b57d714fa087e6bb6e1fe3645a', '809767d7fb8d4c60a2897a219ca6879d']
y
9 578 ['6da594c91927400787b54ebab9b3f4af', 'c8cb43f0bfe043029681133a8ee6a3a5']
y
10 578 ['679e843e11bc4fbcadd2996e7e042a65', 'de8af6c4b5a64d968957f0468c069c4a']
y
11 578 ['94d095cc026f432da30280ad381b5188', 'e1787c031c244a4f9b95598d70ce8b22']

12 578 ['7af03a36ad6c4a0cbaf8

KeyboardInterrupt: 

In [64]:
same_person = same_person[0:-1]

In [69]:
len(same_person)

96

In [96]:
same_person = pickle.load(open('/home/macul/faceRecogOut_same_person_id.pkl','rb'))

In [70]:
pickle.dump(same_person,open('/home/macul/faceRecogOut_same_person_id.pkl','wb'))

In [53]:
df13_m_diff.keys()

Index(['index', 'src_path', 'dst_path_x', 'user_id_x', 'score_x',
       'user_id_top0_x', 'score_top0_x', 'score_top1_x', 'angle_top0_x',
       'angle_top1_x', 'dst_path_top0_x', 'dst_path_top0_name_x',
       'src_path_name_x', 'dst_path_y', 'user_id_y', 'score_y',
       'user_id_top0_y', 'score_top0_y', 'score_top1_y', 'angle_top0_y',
       'angle_top1_y', 'dst_path_top0_y', 'dst_path_top0_name_y',
       'src_path_name_y', 'dst_path_top0_name_diff_sorted'],
      dtype='object')

In [98]:
err_model1=[]
err_model2=[]
img_loc = '/media/macul/black/face_database_raw_data/faceRecog_out_database/var/data/ego_java_server_data/IIM_Images_Crop/{}.jpg'
for idx, row in df12_m_diff.iterrows():
    #print(idx)
    if row['dst_path_top0_name_diff_sorted'] not in same_person:
        print(idx, row['dst_path_top0_name_diff_sorted'])
        img0=Image.open(row['src_path'])
        img0.show()
        img1=Image.open(img_loc.format(row['dst_path_top0_name_1']))
        img1.show()
        img2=Image.open(img_loc.format(row['dst_path_top0_name_2']))
        img2.show()
        xin = input()
        if xin=='1':
            err_model1.append(row['src_path'])
        elif xin=='2':
            err_model2.append(row['src_path'])
        elif xin=='y':
            same_person.append(row['dst_path_top0_name_diff_sorted'])
        else:
            err_model1.append(row['src_path'])
            err_model2.append(row['src_path'])

0 ['84e09aa32b6c4dff9693ec03baf9cc68', '8dcccd3e1b824d80b073fc2e7b731fc0']

1 ['8dcccd3e1b824d80b073fc2e7b731fc0', '904321946cde45499e402c605b686ab4']

2 ['8dcccd3e1b824d80b073fc2e7b731fc0', '904321946cde45499e402c605b686ab4']

3 ['84e09aa32b6c4dff9693ec03baf9cc68', '8dcccd3e1b824d80b073fc2e7b731fc0']

4 ['8dcccd3e1b824d80b073fc2e7b731fc0', '904321946cde45499e402c605b686ab4']

34 ['94d095cc026f432da30280ad381b5188', 'e1787c031c244a4f9b95598d70ce8b22']

35 ['94d095cc026f432da30280ad381b5188', 'e1787c031c244a4f9b95598d70ce8b22']

42 ['8dcccd3e1b824d80b073fc2e7b731fc0', '904321946cde45499e402c605b686ab4']

43 ['8dcccd3e1b824d80b073fc2e7b731fc0', '904321946cde45499e402c605b686ab4']

44 ['8dcccd3e1b824d80b073fc2e7b731fc0', '904321946cde45499e402c605b686ab4']

58 ['285804959b724d5d841c895c63cecf14', 'eb1c8c675c04441890366a2ab3bf5ac0']

59 ['285804959b724d5d841c895c63cecf14', 'fad9a07d51664bd5a02174d2e07b2e9b']

60 ['eb1c8c675c04441890366a2ab3bf5ac0', 'fad9a07d51664bd5a02174d2e07b2e9b']


KeyboardInterrupt: 

In [68]:
print(len(err_model1))
print(len(err_model2))

239
205


In [71]:
err_dic={}
err_dic['faceRecogOut_match_db_dgx_train1'] = err_model2
err_dic['faceRecogOut_match_db_insight_original'] = err_model1
pickle.dump(err_dic, open('/home/macul/model_err.pkl','wb'))

In [7]:
sym_tmp, arg_params, aux_params = mx.model.load_checkpoint(mxModelPath, mxModelEpoch[0])
all_layers = sym_tmp.get_internals()
output_list = all_layers.list_outputs()
sym3 = all_layers['out_embedding_output']
model = mx.mod.Module(symbol=sym3, label_names=None, context=mx.cpu())
model.bind(for_training=False, data_shapes=[('data', (1,3,112,112))])


In [3]:
for i in range(1,len(mxModelEpoch)):
    _, arg_p, aux_p = mx.model.load_checkpoint(mxModelPath, mxModelEpoch[i])
    for k in arg_params.keys():
        arg_params[k] += arg_p[k]
        
    for k in aux_params.keys():
        aux_params[k] += aux_p[k]
        
for k in arg_params.keys():
    arg_params[k] /= len(mxModelEpoch)

for k in aux_params.keys():
    aux_params[k] /= len(mxModelEpoch)

In [8]:
model.set_params(arg_params, aux_params, allow_missing=False, allow_extra=False)

In [10]:
model.save_checkpoint(mxModelPath, mxModelEpoch[0]*100+mxModelEpoch[-1])